# Introduction
### This code is a dummy code that I used to get familiar with downloading meteoswiss data for Tanikon Weather station

- To know the file that should be downloaded daily, one has to go to this link, click on the needed data and then get a link from the end of the wizzard.
https://data.geo.admin.ch/ch.meteoschweiz.ogd-smn/tae/ogd-smn_tae_t_now.csv

- The downloaded dataset contains columns with strange names, the interpretation of these names can be found here:
https://github.com/MeteoSwiss/opendata/blob/main/docs/a-data-groundbased/a1-automatic-weather-stations.md

Namely, these names are coded as follows:
The update interval for this file is around 10 minutes.

Time in UTC: 00:40 UTC = 02:40 local (CH) summer time, 01:40 winter time

Main parameters are:
| Identifier | Description |
|:----------:|:------------|
| `tre200s0` | Air temperature 2 m above ground; current value | 
| `rre150z0` | Precipitation; ten minutes total |
| `sre000z0` | Sunshine duration; ten minutes total |
| `gre000z0` | Global radiation; ten minutes mean |
| `ure200s0` | Relative air humidity 2 m above ground; current value |
| `tde200s0` | Dew point 2 m above ground; current value |
| `dkl010z0` | Wind direction; ten minutes mean |
| `fu3010z0` | Wind speed; ten minutes mean in km/h |
| `fu3010z1` | Gust peak (one second); maximum in km/h |
| `prestas0` | Atmospheric pressure at barometric altitude (QFE<sup>1</sup>); current value |
| `pp0qffs0` | Pressure reduced to sea level (QFF<sup>2</sup>); current value |
| `pp0qnhs0` | Pressure reduced to sea level according to standard atmosphere (QNH<sup>3</sup>); current value |
| `ppz850s0` | Geopotential height of the 850 hPA level; current value |
| `ppz700s0` | Geopotential height of the 700 hPA level; current value |
| `dv1towz0` | Wind direction vectorial tower<sup>4</sup>; ten minutes mean |
| `fu3towz0` | Wind speed scalar tower<sup>4</sup>; ten minutes mean in km/h |
| `u3towz1` | Gust peak (one second) tower<sup>4</sup>; maximum in km/h |
| `ta1tows0` | Air temperature tower<sup>4</sup>; current value |
| `uretows0` | Relative air humidity tower<sup>4</sup>; current value |
| `tdetows0` | Dew point tower<sup>4</sup>; current value |

<sup>1</sup> QFE = the isobaric surface pressure at the reference point

<sup>2</sup> QFF = the air pressure at the measuring site, reduced to the sea level. It is used in the field of meteorology in order to ­compare the air pressures of different places at different heights.

<sup>3</sup> QNH = the air pressure at the measuring station, reduced to sea level acc. to "standard atmosphere".

<sup>4</sup> Information about the [automatic tower stations](/a-data-groundbased/a3-automatic-tower-stations) dataset/collection.


In [ ]:
# Import necessary libraries
import pandas as pd  # Data processing: https://pandas.pydata.org/docs/
import matplotlib.pyplot as plt  # Visualization: https://matplotlib.org/stable/contents.html
import seaborn as sns  # Also used for visualization: https://seaborn.pydata.org/
import pandas as pd
import requests
import io
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)


#URL to get meteoswiss data
url_data = "https://data.geo.admin.ch/ch.meteoschweiz.ogd-smn/tae/ogd-smn_tae_t_now.csv"

def download_csv(url):
    """
    Downloads a CSV file from a given URL (skipping SSL verification)
    and returns a Pandas DataFrame.
    """
    try:
        response = requests.get(url, verify=False, timeout=30)  # verify=False skips SSL checks
        response.raise_for_status()  # raise exception if status not 200
        df = pd.read_csv(io.BytesIO(response.content), delimiter=';')
        return df
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

In [ ]:
def build_param_map(df_metadata: pd.DataFrame) -> dict:
    # Keep only the 1st and 5th columns by position
    sub = df_metadata.iloc[:, [0, 4]].dropna()
    sub.columns = ["parameter", "parameter_description_en"]
    mapping =  dict(zip(sub["parameter"].str.strip(), sub["parameter_description_en"].str.strip()))
    return mapping

df_metadata = pd.read_csv('ogd-smn_meta_parameters.csv', sep=";", encoding="latin1", dtype=str)
mapping = build_param_map(df_metadata)
mapping



In [ ]:
import re

df_data = download_csv(url_data)

mapping = {c: mapping[c] for c in df_data.columns if c in mapping}


def clean_name(s: str) -> str:
    # keep only before first ';'
    s = s.split(";")[0]
    # lowercase
    s = s.lower()
    # replace non-alphanumeric with underscores
    s = re.sub(r'[^0-9a-zA-Z]+', '_', s)
    # strip leading/trailing underscores
    return s.strip("_")

# apply to your mapping
mapping = {k: clean_name(v) for k, v in mapping.items()}
mapping


In [ ]:
data_final = df_data.rename(columns=mapping)
data_final